<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/conv_specification_example/tests/conv_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnx onnxruntime

In [2]:
from onnx import TensorProto
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model

In [32]:
# @title Standard max pooling (1 channel)
import numpy as np
from onnx import *
import onnxruntime

# Create inputs
x = helper.make_tensor_value_info('x', TensorProto.DOUBLE, [1, 1, 8, 8])

# Create a node (MaxPool) with input/outputs
node_def = make_node(
    'MaxPool', # node name
    ['x'], # inputs
    ['y', 'Indices'], # outputs
    dilations=[1,1],
    kernel_shape=[3,3],
    pads=[0, 0, 0, 0],
    strides=[1, 1],
    auto_pad='NOTSET',
#    group=1, # Standard convolution
)

# Create the graph
graph_def = make_graph(
    [node_def],
    'test-maxpool',
    [x],
    [helper.make_tensor_value_info('y', TensorProto.DOUBLE, [1, 1, 6, 6]), 
     helper.make_tensor_value_info('Indices', TensorProto.INT64, [1, 1, 6, 6])],
)

onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Verify the model
check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Do inference
sess = onnxruntime.InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

# Initialize tensors
#x = numpy.ones((1, 1, 8, 8), dtype=numpy.float32)

x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))

y = sess.run(None, {'x': x})[0]
Indices = sess.run(None, {'x': x})[1]

print("X shape:", x.shape)
print("X:", x)
print("Y shape:", y.shape)
print("Y:", y)
print("Indices shape:", Indices.shape)
print("Indices:", Indices)


graph test-maxpool (
  %x[DOUBLE, 1x1x8x8]
) {
  %y, %Indices = MaxPool[auto_pad = 'NOTSET', dilations = [1, 1], kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [1, 1]](%x)
  return %y, %Indices
}
X shape: (1, 1, 8, 8)
X: [[[[ 5.67591154 -0.04859958  2.94203104  3.70327292  2.47014306
     4.12455586  5.81838665  1.84118807]
   [-0.05267874  2.75227858  2.16608732  4.03416243  1.28184638
     4.81748948  4.64878412  3.31626988]
   [ 3.55427648  0.39997585  4.45761508  4.82722666  0.18843372
     0.49564314  7.96647029  4.82851447]
   [ 1.52417623  2.28965587  0.36251913  1.64413983  4.67267459
     3.73167179  2.20052118  2.06720836]
   [-1.22446366 -0.86469519  6.01461967 -1.08813165  2.11920055
     0.78561867  0.29834533  1.94499626]
   [ 1.57776732  3.64260188  3.47181319  4.83723727  1.49868674
     3.27683692  2.42625178  0.4401565 ]
   [ 6.8972704   5.51113868  5.99293336  4.24088721  1.94993561
    -0.04040625  3.07940675  3.06769141]
   [ 3.1299626   4.5546675   3.500819